# A quick insight on Algebra and KZG Commitments

_(These contents do not attempt to be exhaustive. The main objective is to give the bare minimum background to understand KZG commitments)_
 
_**Disclaimer**: lack of formalism ahead!_

## Contents

1. [Preliminaries](#1.-Preliminaries)
    - 1.1 [Number Sets](#1.1-Number-Sets)
    - 1.2 [Prime Numbers](#1.2-Prime-Numbers)
    - 1.3 [Coprime Integers](#1.3-Coprime-Integers)
    - 1.4 [Modular aritmethic](#1.4-Modular-aritmethic)
    - 1.5 [Basic Algebraic Structures](#1.5-Basic-Algebraic-Structures)
        - 1.5.1 [Abelian Groups](#1.5.1-Abelian-Groups)
            - [Finite Groups, Cyclic Groups and Generators](#Finite-Groups,-Cyclic-Groups-and-Generators)
            - [The Exponential Map](#The-Exponential-Map)
            - [Pairings](#Pairings)
        - 1.5.2 [Rings](#1.5.2-Rings)
        - 1.5.3 [Fields & Galois Fields](#1.5.3-Fields-&-Galois-Fields)
    - 1.6 [Cryptographic Groups & Computational Hardness Assumptions](#1.6-Cryptographic-groups-&-computational-hardness-assumptions)
    - 1.7 [Polynomials](#1.7-Polynomials)
2. [KZG Commitments](#2.-KZG-Commitments)
    - 2.1 [The Setup Phase](#2.1-The-Setup-Phase)
    - 2.2 [The Commitment Phase](#2.2-The-Commitment-Phase)
    - 2.3 [The Opening Phase](#2.3-The-Opening-Phase)
    - 2.4 [The Verification Phase](#2.4.-The-Verification-Phase)
    - 2.5 [A `sagemath` Implementation](#2.5-A-sagemath-Implementation)
3. [Closing remarks](#3.-Closing-remarks)
    - 3.1 [Recovering the message from the polynomial?](#3.1-Recovering-the-message-from-the-polynomial?)
    - 3.2 [Abstraction!](#3.2-Abstraction!)

# 1. Preliminaries

One can define Mathematics as the field that studies patterns, as it contains a vast amount of subfields, each of which ultimately leads to the description and formalization of patterns and behaviors under the formalism of axioms and theorems. In fact, the whole Mathematic universe sustains itself on a series of axioms (statements that are taken as true), being the most widely used the [Zermelo-Fraenkel axioms](https://en.wikipedia.org/wiki/Zermelo%E2%80%93Fraenkel_set_theory) (ZF axioms), which are the basis of modern Set Theory.

It is through set theory that we can define the most basic mathematical object: numbers.

## 1.1 Number Sets

We are going to start with the set of Natural numbers $\mathbb{N}$, which most authors define as the set of all positive integers: $\{1,2,3,4,\dots\}$ (we denote a set as a pair of $\{\}$ containing some/all of the elements of the set). If you're curious about how this set is constructed, check out [Wikipedia's article on the Set theoretic definition of natural numbers](https://en.wikipedia.org/wiki/Set-theoretic_definition_of_natural_numbers). Some mathematicians (like myself) would also include the number $0$ in this set, but for the sake of coherency with Least Authority's MoonMath manual, we'll stick to the former definition. In case one would want to explicitly include $0$ in the set of Natural numbers, some authors adopt the notation $\mathbb{N}_0$.

The set of Integers $\mathbb{Z}$ is an extension of $\mathbb{N}$ and is defined as the set of all positive and negative integers: $\{\dots,-3,-2,-1,0,1,2,3,\dots\}$. The need of this extension is due to the fact that some operations (like subtraction $^*$) are not closed under $\mathbb{N}$, meaning that the result of the operation is not necessarily an element of $\mathbb{N}$. More on that later.


The set of Rational numbers $\mathbb{Q}$ is an extension of $\mathbb{Z}$ and is defined as the set of all numbers that can be expressed as the quotient of two integers (assuming the divisor is non-zero): $\{\frac{a}{b} \mid a,b\in\mathbb{Z}, b\neq 0\}$ (this reads like _"the set composed by the quotients of $a$ and $b$, so that $a$ and $b$ are elements of the set of integers and $b$ is non-zero"_). This extension also arises from the need of closure under some operations (like division $^*$).

We are going to define the set of Irrational numbers (some authors denote it as $\mathbb{I}$) and the set of Real numbers $\mathbb{R}$ together. $\mathbb{I}$ is defined as the set of numbers that _"cannot be expressed as the quotient of two integers"_, and $\mathbb{R}$ can be easily defined as the union of $\mathbb{Q}$ and $\mathbb{I}$: $\mathbb{R} = \mathbb{Q}\cup\mathbb{I}$. Given that we won't need these number sets for this session, we won't touch on the construction of $\mathbb{R}$ and $\mathbb{I}$. If curious, you may check out [Wikipedia's article on the construction of the real numbers](https://en.wikipedia.org/wiki/Construction_of_the_real_numbers) and [Wikipedia's article on irrational numbers](https://en.wikipedia.org/wiki/Irrational_number).

Finally, another set that is of general interest in mathematics is the set of Complex numbers $\mathbb{C}$, which is defined as the set of numbers that can be expressed as $a+bi$, where $a,b\in\mathbb{R}$ and $i$ is the imaginary unit, defined as $i^2=-1$. This set extends the real numbers $\mathbb{R}$ and arises from the need of closure under some operations (like the square root of a negative number $^*$)

$^*$ _Soon you wil see that "subtraction", "division" and "square root" are actually aliases for something else..._ 😯

From now on, we'll be working with the set of Integers $\mathbb{Z}$ and the set of Natural numbers $\mathbb{N}$, so don't worry too much about the other sets.

## 1.2 Prime Numbers

A prime number is a natural Number ($\mathbb{N}$) that has the following properties:

- is greater than 1
- is divisible only by itself and $1$

_Remember: $a$ is divisible by $b$ if_ $\frac{a}{b}\in\mathbb{Z}$

We denote the set of prime numbers as $\mathbb{P}$. Note that there is only one even prime number: $2$. Several authors denote $\mathbb{P}_{\geq 3}$ as the set of **odd** prime numbers.

## 1.3 Coprime Integers

It is said that two integers $a$ and $b$ are coprime if their greatest common divisor (GCD) is $1$. We denote this as $gcd(a,b)=1$. This means that $a$ and $b$ do not share any common factors other than $1$.

For example $gcd(4,9)=1$ ($4$ and $9$) are coprime, but $gcd(4,8)=4$ ($4$ and $8$ are not coprime). Usually, this is checked by means of the [Euclidean algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm). A more brute-force way to check if two numbers are coprime is to check if their prime factorization has any common factors. Let's take the previous examples:

- $4$ and $9$ are coprime: $4=2^2$ and $9=3^2$, so they don't share any common factors
- $4$ and $8$ are not coprime: $4=2^2$ and $8=2^3$ because they share the common factor $2$ twice! $4=2^2$ and $8=2^3=2\cdot 2^2$, thus the GCD is $2^2=4$

A fun corollary of this is that if either $a$ or $b$ is prime, then $gcd(a,b)=1$, i.e. the pair is co-prime. This is because a prime number can only be divided by itself and $1$.

## 1.4 Modular aritmethic

Modular arithmetic is a system of arithmetic for integers, where numbers "wrap around" upon reaching a certain value—the modulus _(doesn't this sound familiar with computer science's concept of integer overflow/underflow? A Solidity `uint256` type is a positive integer defined on modular arithmetic of modulus $2^{256}$ !_ 🤯 _)_

For example, let's take modulus 5, and add the numbers 4 and 3:

$$4+3=7\equiv 2\quad\text{mod}~5$$

_(Note that the symbol $\equiv$ means "equivalent")_

So basically, $4$ plus $3$ is $7$, but if we "wrap around" the result to the modulus $5$, we get $2$. Another way to think about it is that every time we get a number that is greater than the modulus, we subtract the modulus until we get a number that is less than the modulus. i.e. we divide by the modulus and keep the remainder.

The exact same thing happens with multiplication:

$$2\cdot 4=8\equiv 3\quad\text{mod}~5$$

One can quickly notice that the modulus gives the number of possible results of the operation. In the previous example, we had $5$ possible results: $0,1,2,3,4$. So, if we are in modulus $n$ the set of numbers $\{0,\dots,n-1\}$ are actually all the numbers we will ever get when doing arithmetic operations with modulus $n$. This set is denoted as $\mathbb{Z_n}$ (other mathematicians, as myself, prefer denoting it as $\mathbb{Z}/n\mathbb{Z}$).

These concepts give rise to **equivalence classes**. To put it easy, an equivalente class is a set of numbers that _"are equal"_ under modulus $n$. From each equivalente class, we take _"one representative number"_ to identify each class. Let's go back again to our modulus $5$ example. As mentioned, $\mathbb{Z}_5=\{0,1,2,3,4\}$. So, the equivalence class of $0$ is $\{\dots,-5,0,5,10,15,\dots\}$, basically $\{a\mid a\in\mathbb{Z}, \frac{a}{5}\in\mathbb{Z}\}$. What about the equivalence class of $3$? Well, it's $\{\dots,-2,3,8,13,18,\dots\}$, basically $\{a\mid a\in\mathbb{Z}, \frac{a - 3}{5}\in\mathbb{Z}\}$. It's the set of numbers that _"fall"_ on the same number once wrapping around 5 (i.e. the set of numbers that have the same remainder when divided by 5).

If the equivalence classes remain unclear, think about a clock: it has $12$ hours, and once it reaches $12$, it goes back to $1$. So, the equivalence class of $1$ is $\{\dots,1,13,25,\dots\}$, the equivalence class of $2$ is $\{\dots,2,14,26,\dots\}$, and so on. The equivalence class of $12$<> is $\{\dots,0, 12,24,36,\dots\}$. That's why when a digital clock reads `13:00`, we say it's `1:00 PM`, because $13$ is equivalent to $1$ under modulus $12$.

These sets will give rise to interesting algebraic structures, as we'll see later.

## 1.5 Basic Algebraic Structures

So far, we've seen some basic number sets and some basic arithmetic operations. Now, we'll see some basic algebraic structures that arise from these sets and operations.

But first, what is Algebra? Many people think of Algebra as the study of equations, but that's not entirely true. Algebra is the study of mathematical structures, their properties and how to manipulate them. To put it simple, it is like having a _"bag of objects"_ and some _"rules"_ to manipulate those _"objects"_.

_(Doesn't this ring a bell for you? It's like having a `class` in Object Oriented Programming, and some `methods` to manipulate the `class` `instance` !_ 🤯 _)_

### 1.5.1 Abelian Groups

Any algebraic structure needs a set and an operation. Let's start with the most basic algebraic structure: the Abelian Group. An Abelian Group (also known as commutative group) $\mathbb{G}$ is a tuple $(G,\oplus)$ where $G$ is a set and $\oplus$ is a binary operation (i.e. an operation that takes two elements of the set and returns another element) that satisfies the following properties:

- Closure: $\forall a,b\in G, a\oplus b\in G$ (the elements returned by the operation are elements of the set).
- Associativity: $\forall a,b,c\in G, (a\oplus b)\oplus c=a\oplus(b\oplus c)$
- Identity element: $\exists e\in G, \forall a\in G, a\oplus e=e\oplus a=a$. $e$ is called the identity element.
- Inverse element: $\forall a\in G, \exists a^{-1}\in G, a\oplus a^{-1}=a^{-1}\oplus a=e$. $a^{-1}$ is called the inverse element of $a$.
- Commutativity: $\forall a,b\in G, a\oplus b=b\oplus a$

If you're familiar with Object Oriented Programming, you can think of an Abelian Group as a `class` with a `method` that takes two `instances` of the `class` and returns another `instance` of the `class`. The `class` is the set $G$, the `method` is the binary operation $\oplus$, and the `instances` are the elements of the set $G$.

Let's see some examples of Abelian Groups:

- The set of integers $\mathbb{Z}$ with the binary operation $+$ (addition) is an Abelian Group. The identity element is $0$ and the inverse element of $a$ is $-a$. _(See why [earlier](#1.1-Number-Sets) we mentioned that subtraction is an alias? It is an alias for "addition with the inverse")_
- The set of integers $\mathbb{Z}$ with the binary operation $\cdot$ (product; multiplication) is **NOT** an Abelian Group. The identity element is $1$ but the inverse element of $a$ is $\frac{1}{a}$, which is NOT an element of $\mathbb{Z}$. _(Again, see why [earlier](#1.1-Number-Sets) we mentioned that division is an alias? It is an alias for "multiplication with the inverse")_.
- The set of rational numbers $\mathbb{Q}$ with the binary operation $\cdot$ (product; multiplication) is an Abelian Group. The identity element is $1$ and the inverse element of $a$ is $\frac{1}{a}$. _(See why [earlier](#1.1-Number-Sets) we mentioned that $\mathbb{Q}$ is an extension of $\mathbb{Z}$ ?)_

If the commutativity property is not present in the Abelian Group, then it is just called a Group. _Can you think of an example of a tuple $(G,\oplus)$ that is a Group but not an Abelian Group?_ 😉

#### Finite Groups, Cyclic Groups and Generators

#### The Exponential Map

#### Pairings

### 1.5.2 Rings

### 1.5.3 Fields & Galois Fields

## 1.6 Cryptographic groups & computational hardness assumptions

## 1.7 Polynomials


# 2. KZG Commitments

## 2.1 The Setup Phase

## 2.2 The Commitment Phase

## 2.3 The Opening Phase

## 2.4. The Verification Phase

## 2.5 A `sagemath` Implementation

Let's replicate (and implement) the previous steps one by one.

> Remember, we want to prove that we know a Polynomial $\phi(x) = a_0 + a_1x + a_2x^2+\dots+a_tx^t$, where $\phi(x)\in\mathbb{F}_p[x]$.

### 2.5.1 Setup

First we'll have to define a couple variables:
- $p$ prime which will aid us to build our Prime Field $\mathbb{F}_p$
- $t$ the maximum degree of the polynomials in our Polynomial Field $\mathbb{F}_p[x]$, so that $t < p$

Remember that our prime needs $k$ bits of security! So we'll have to define a function to get a safe random prime number

In [1]:
def safe_random_prime(k):
    """Generate a safe random prime.
    Code taken from
    https://ask.sagemath.org/question/44765/randomly-generate-a-safe-prime-of-given-length/?answer=44766#post-id-44766
    """
    while true:
        p = random_prime(2^k - 1, false, 2^(k - 1))
        if ZZ((p - 1)/2).is_prime():
            return p

k = 16 # 16 bits of security. My laptop is not beefy and I want my public parameters to be computed in a matter of seconds
p = safe_random_prime(k)
print(f"Our safe random prime is {p}")

Our safe random prime is 39779


Let's define the Prime Field $\mathbb{F}_p$

In [2]:
F_p = GF(p)
print(F_p)

Finite Field of size 39779


Let's generate a random $t$ so that $t < p$

In [3]:
t = 0

while t == 0:
    t = F_p.random_element()
print(f"The maximum degree is {t}")

The maximum degree is 32983


Now, let's get the generator $g$ of the Prime Field $\mathbb{F}_p$

In [4]:
g = F_p.multiplicative_generator()
print(f"The generator is {g}")

The generator is 2


Next, we have to define a pairing $e:\mathbb{G}\times\mathbb{G}\rightarrow\mathbb{G}_T$. We will define the simple pairing
$$
e(g_1,g_2)\mapsto g_1\cdot g_2 \quad\left(\text{mod}~p\right)
$$

In [5]:
def e(g1, g2):
    return F_p.prod((g1, g2))

We may also check that this pairing
- is bilinear:
$$
e(g_1^a,g_2^b)=e(g_1^{ab},g_2)=e(g_1,g_2^{ab})=e(g_1,g_2)^{ab}
$$
    see:
$$
ag_1\cdot bg_2 = abg_1\cdot g_2 = g_1\cdot abg_2 = ab(g_1\cdot g_2) \quad\left(\text{mod}~p\right)
$$ 
- non-degenerate
$$
e(g,g)\neq\boldsymbol{1}
$$
    see:
$$
2\cdot2 = 4 \neq 0 \quad\left(\text{mod}~p\right)
$$

At this point, the trusted party (us, haha) retrieves a random (and secret) $\alpha\in\mathbb{F}_p$

In [6]:
alpha = F_p.random_element()

This allows computing the Public Parameters `PP`

In [7]:
def compute_public_parameters(F_p, alpha, t):

    PP = []

    accumulated = 1
    for i in range(t + 1):
        PP.append(F_p.prod((
            accumulated, g
        )))
        accumulated = F_p.prod((accumulated, alpha))
    return PP

In [8]:
PP = compute_public_parameters(F_p, alpha, t)
print("The first elements of the public parameters are: {}".format(PP[:100]))

The first elements of the public parameters are: [2, 33754, 30978, 309, 3944, 12621, 27851, 12663, 20663, 26737, 27152, 30003, 13740, 18189, 1210, 14543, 5781, 27829, 39159, 37916, 23338, 23547, 10509, 25611, 38012, 12591, 38668, 25341, 16028, 7356, 36732, 10028, 22690, 26476, 37724, 5053, 33084, 20624, 4998, 19766, 4088, 16390, 30643, 34911, 26178, 20532, 3695, 26822, 29653, 33861, 6983, 26693, 586, 24730, 6942, 10979, 2001, 38285, 5648, 10812, 7851, 37257, 39515, 39499, 8141, 38770, 36298, 4746, 23115, 38981, 17235, 11047, 35825, 17504, 16154, 25571, 39175, 29495, 32488, 26019, 2282, 7242, 22146, 34337, 5077, 563, 34349, 8706, 27315, 36203, 32370, 23483, 4414, 28790, 28124, 5720, 32586, 9247, 8602, 22383]


Remember, once we have the public parameters, **we must delete `alpha`**.
_We are not going to do it though, for illustrative purposes!_

### 2.5.2 Commitment

It's time to do our commitment! In order to make it as applied as possible, let's commit a message. To do that though, we will some auxiliary code to help us!

#### Reed Solomon Codes

We're going to use Reed-Solomon encoding to transform our messages into a list of integers

In [10]:
from reedsolo import RSCodec

def reed_solomon_encode_string(input_string):

    # Initialize the Reed-Solomon encoder/decoder with a specific error correction level
    rs = RSCodec(10)  # You can adjust the error correction level as needed

    # Encode the input string
    encoded_bytes = rs.encode(input_string.encode())

    # Convert the encoded bytes to a list of integers
    return list(encoded_bytes)

We will also need to transform this into a list of tuples, which will contain coordinates

In [11]:
def message_as_coords(input_string):
    y = reed_solomon_encode_string(input_string)
    x = range(1, len(y) + 1)
    return list(zip(x, y))

Let's encode a message!

In [12]:
msg = message_as_coords("Hello Zepp!")
print(msg)

[(1, 72), (2, 101), (3, 108), (4, 108), (5, 111), (6, 32), (7, 90), (8, 101), (9, 112), (10, 112), (11, 33), (12, 25), (13, 255), (14, 128), (15, 142), (16, 165), (17, 163), (18, 57), (19, 193), (20, 169), (21, 195)]


#### Interpolating Polynomial
Cool! Let's create our Polynomial Ring $\mathbb{F}_p[x]$

In [13]:
F_p_X.<x> = PolynomialRing(F_p)
F_p_X

Univariate Polynomial Ring in x over Finite Field of size 39779

And get an interpolating polynomial over our coordinates (our message)

In [14]:
msg_poly = F_p_X.lagrange_polynomial(msg)
msg_poly

20307*x^20 + 2518*x^19 + 1520*x^18 + 34792*x^17 + 29777*x^16 + 25961*x^15 + 34636*x^14 + 11875*x^13 + 13526*x^12 + 34297*x^11 + 29923*x^10 + 31332*x^9 + 6401*x^8 + 19111*x^7 + 20394*x^6 + 37056*x^5 + 6861*x^4 + 24873*x^3 + 11595*x^2 + 4072*x + 36814

Since for $n$ points $(x,y)$ there exists one unique polynomial of degree $n-1$ that passes through all of them, we basically converted `"Hello Zep"` into a polynomial!

---

Okay, now we have the polynomial that we want to commit. We will need to evaluate the polynomial at $\alpha$. We will "_evaluate it on the exponent_". We can write a function for that

In [15]:
from functools import reduce

def exponent_evaluation(PP, poly, g):
    # Recover the Prime Field
    F_p = poly.base_ring()
    # Get the Polynomial coefficients. They are given in ascending order
    poly_coefs = poly.coefficients()
    # Only keep the parameters needed
    pp_used = PP[:len(poly_coefs)]  
    
    return reduce(
        lambda a,b: a + (b[0]*b[1]),
        zip(poly_coefs, pp_used),
        0
    )

In [16]:
commitment = exponent_evaluation(PP, msg_poly, g)
commitment

16202

This is our commitment!

### 2.5.3 Opening

The Verifier asks the Prover to evaluate the Polynomial at a point $i\in\mathbb{F}_p$. So, let's choose this point randomly

In [17]:
i = F_p.random_element()
print(f"The point of evaluation `i` is {i}")

The point of evaluation `i` is 1220


Now we can evaluate our polynomial on `i`

In [18]:
poly_eval = msg_poly(i)
print(f"The evaluation is {poly_eval}")

The evaluation is 10371


Next we compute the proof polynomial

In [19]:
proof_poly = ((msg_poly - poly_eval)/(x - i)).numerator()
proof_poly

20307*x^19 + 34520*x^18 + 29738*x^17 + 36704*x^16 + 17503*x^15 + 18298*x^14 + 2398*x^13 + 33568*x^12 + 33895*x^11 + 16037*x^10 + 23795*x^9 + 22562*x^8 + 4973*x^7 + 39763*x^6 + 874*x^5 + 29303*x^4 + 34979*x^3 + 16386*x^2 + 33457*x + 8358

And the corresponding opening commit

In [20]:
opening_commit = exponent_evaluation(PP, proof_poly, g)
opening_commit

27985

Now we have the opening triplet $\left(i,\phi(i),\mathcal{C}^{\psi_i(\alpha)}\right)$

In [21]:
opening_triplet = (i, poly_eval, opening_commit)
opening_triplet

(1220, 10371, 27985)

### 2.5.4 Verification

Now the Verifier has to check if $\phi(i)$ is indeed evaluating $\phi(x)$ on $i$

In [22]:
e(commitment, g) == (e(opening_commit, PP[1] - i*g) + (poly_eval*e(g,g)))

True

### 2.5.5 What if $\alpha$ is leaked?
In that case, we could forge a polynomial that passes the proof without knowing the message at all...
In fact, we have not deleted `alpha`, so we can actually act as bad actors and forge said polynomial!

In [23]:
print(f"Oh no, `alpha` leaked: {alpha}")

Oh no, `alpha` leaked: 16877


So we know the original commitment:

In [24]:
print(f"Original commitment: {commitment}")

Original commitment: 16202


Since the commitment is $g^{\phi(\alpha)}$, and $g$ is publicly known, we can also know a value for $\phi(\alpha)$

In [25]:
commitment/g

8101

Now we just need a polynomial that passes through $(\alpha, \phi(\alpha))$... Just to make it nice, let's compute it as a quadratic function

In [26]:
def make_faux_poly(F_p_X, alpha, phi_alpha, i, phi_i):
    xs = [alpha, i]
    ys = [phi_alpha, phi_i]

    F_p = F_p_X.base_ring()

    while (len(xs) != 3):
        x = F_p.random_element()
        if not (x in xs):
            xs.append(x)
            ys.append(F_p.random_element())
    return F_p_X.lagrange_polynomial(zip(xs,ys))

In [27]:
faux_poly = make_faux_poly(F_p_X, alpha, commitment/g, i, poly_eval)
faux_poly

32674*x^2 + 7560*x + 9865

Let's run the protocol with this faux polynomial that generates an identical commitment

In [28]:
faux_poly_eval = faux_poly(i)
print(f"The evaluation of i on the faux polynomial is {faux_poly_eval}")

faux_proof_poly = ((faux_poly - faux_poly_eval)/(x - i)).numerator()
print(f"The faux proof polynomial {faux_proof_poly}")

faux_opening_commit = exponent_evaluation(PP, proof_poly, g)
opening_triplet = (i, faux_poly_eval, faux_opening_commit)
print(f"The faux opening triplet is {opening_triplet}")

faux_verification = e(commitment, g) == (e(faux_opening_commit, PP[1] - i*g) + (faux_poly_eval*e(g,g)))
print(
f"""
Did we deceive the system and verified the faux polynomial {faux_poly}
as if it was the true polynomial {msg_poly}?\n\n{faux_verification}!!
"""
)

The evaluation of i on the faux polynomial is 10371
The faux proof polynomial 32674*x + 11282
The faux opening triplet is (1220, 10371, 27985)

Did we deceive the system and verified the faux polynomial 32674*x^2 + 7560*x + 9865
as if it was the true polynomial 20307*x^20 + 2518*x^19 + 1520*x^18 + 34792*x^17 + 29777*x^16 + 25961*x^15 + 34636*x^14 + 11875*x^13 + 13526*x^12 + 34297*x^11 + 29923*x^10 + 31332*x^9 + 6401*x^8 + 19111*x^7 + 20394*x^6 + 37056*x^5 + 6861*x^4 + 24873*x^3 + 11595*x^2 + 4072*x + 36814?

True!!



# 3. Closing remarks

## 3.1 Recovering the message from the polynomial?

Did you wonder how can we recover the original message from the polynomial?

In [29]:
def decode_poly(poly):
    images = []
    for i in range(1, poly.degree() + 2):
        images.append(poly(i))

    rs = RSCodec(10)
    
    return bytes(rs.decode(images)[0]).decode("utf-8")

In [30]:
decode_poly(msg_poly)

'Hello Zepp!'

Ever wondered what our faux polynomial was encoding?

In [31]:
try:
    print(decode_poly(faux_poly))
except:
    print("Exception!\nEncoded data makes no sense as a reed-solomon encoding")

Exception!
Encoded data makes no sense as a reed-solomon encoding


## 3.2 Abstraction!

Let's abstract all of this into a class

In [32]:
from KZG import KZG

Cool! Let's do all of the previous work again

In [33]:
# Initialize the helper class
kzg = KZG(leak=True)
kzg

KZG commitment class with Finite Field of size 46679 whose generator is 17
Toxic waste has NOT been deleted. Proceed with caution

Let's create the polynomial encoding our message `"Hello Zepp"` and commit it to the KZG proving scheme

In [34]:
msg = message_as_coords("Hello Zepp!")
msg_poly_again = kzg.polynomial_ring()[0].lagrange_polynomial(msg)
commitment_again = kzg.make_commitment(msg_poly_again)
commitment_again

44188

Let's create now the opening triplet. We'll choose an arbitrary `i`

In [35]:
i_again = kzg.F_p.random_element()
i_again

36723

In [36]:
opening_triplet_again = kzg.make_opening_triplet(i_again, msg_poly_again)
opening_triplet_again

(36723, 24341, 33270)

Let's verify it!

In [37]:
kzg.verify(commitment_again, opening_triplet_again)

True

Let's forge a proof again

In [38]:
faux_poly_again = make_faux_poly(
    kzg.polynomial_ring()[0],
    kzg.alpha,
    commitment_again/kzg.g,
    *opening_triplet_again[:2]
)
faux_poly_again

1088*x^2 + 30264*x + 41025

In [39]:
faux_commitment_again = kzg.make_commitment(faux_poly_again)
faux_commitment_again

44188

In [40]:
faux_opening_triplet = kzg.make_opening_triplet(i_again, faux_poly_again)
faux_opening_triplet

(36723, 24341, 33270)

In [41]:
kzg.verify(faux_commitment_again, faux_opening_triplet)

True